# Augmentation system for CV ML algorithms

### Specification:
Using Python and OpenCV write a program that:
- Allows user to select a directory on local disk. (e.g. using tkinter library)
- Read all .jpg images from this directory and, for each of them, apply a set of predefined augmentation algorithms with a set of predefined parameters. (e.g. Rotation with +15 degree).
- The augmentation algorithms and corresponding parameters to be applied will be loaded when the program starts from a configuration file (plain text, xml etc.)
- The results of augmentation process will be saved on a new directory (output dir), having the same name with the original one plus the "_aug" suffix.
- Each augmented image will be saved in the output dir having the name of augmentation algorithm as suffix followed by an incremental number starting with "_1".

In [87]:
# all imports
import tkinter as tk
from tkinter import filedialog
import json
import cv2
import os
import numpy as np

In [88]:
class AugmentationTechnique:
    def __init__(self, augmentation_type, number1, number2, color):
        self.augmentation_type = augmentation_type
        self.number1 = number1
        self.number2 = number2
        self.color = color

    def __str__(self) -> str:
        string = self.augmentation_type + " "
        if self.number1:
            string += self.number1 + " "
        if self.number2:
            string += self.number2 + " "
        if self.color:
            string += self.color + " "
        return string


In [89]:
class AugmentationSystem:
    def __init__(self, augmentation_array, file_path):
        """
        Constructor for the AugmentationSystem class.

        Parameters:
        - augmentation_array: A list of lists, each inner list represents a sequence of augmentations.
        - file_path: The path to the folder containing the input images.
        """
        self.augmentation_array = augmentation_array
        self.file_path = file_path
        self.count = 1
        self.images = self.load_images()
        self.prepare_destination_folder()

    def load_images(self):
        """
        Load images from the specified folder and return them as a dictionary.

        Returns:
        - images: A dictionary where keys are file names and values are loaded images (NumPy arrays).
        """
        image_files = [f for f in os.listdir(self.file_path) if os.path.isfile(os.path.join(self.file_path, f))]
        images = {}
        for file in image_files:
            file_path = os.path.join(folder_path, file)
            image = cv2.imread(file_path)
            if image is not None:
                images[file] = image
            else:
                print(f"Failed to load image: {file_path}")
        return images

    def prepare_destination_folder(self):
        """
        Prepare the destination folder for augmented images. If it doesn't exist, create it.
        If it does exist, remove its contents.
        """
        if not os.path.exists(self.file_path + "_aug"):
            os.mkdir(self.file_path + "_aug")
        else:
            items = os.listdir(self.file_path + "_aug")
            for item in items:
                item_path = os.path.join(self.file_path + "_aug", item)
                os.remove(item_path)

    def perform_augmentations(self):
        """
        Apply the specified augmentations to the loaded images and save the augmented images to the destination folder.
        """
        for augmentation_list in self.augmentation_array:
            for file, image in self.images.items():
                file_name = file.split(".")[0] + "_"
                new_image = image.copy()
                for aug in augmentation_list:
                    file_name += aug.augmentation_type + "_"
                    if aug.augmentation_type == "Blur":
                        new_image = self.blur(new_image)
                    elif aug.augmentation_type == "Sharpen":
                        new_image = self.sharpen(new_image)
                    elif aug.augmentation_type == "Tint":
                        new_image = self.tint(new_image, int(aug.number1), aug.color)
                    elif aug.augmentation_type == "Flip":
                        new_image = self.flip(new_image, int(aug.number1))
                    elif aug.augmentation_type == "Rotate":
                        new_image = self.rotate(new_image, int(aug.number1))
                    elif aug.augmentation_type == "Translate":
                        new_image = self.translate(new_image, int(aug.number1), int(aug.number2))

                cv2.imwrite(self.file_path + "_aug/" + file_name + str(self.count) + ".jpg", new_image)
                print("Saved image: " + self.file_path + "_aug/" + file_name + str(self.count) + ".jpg")
                self.count += 1

    @staticmethod
    def blur(image):
        """
        Apply a Gaussian blur to the input image.

        Parameters:
        - image: The input image.

        Returns:
        - blurred_image: The image with Gaussian blur applied.
        """
        kG = (1/16) * np.array([[1,2,1], [2, 4, 2], [1, 2, 1]])
        return cv2.filter2D(image, -1, kG)

    @staticmethod
    def sharpen(image):
        """
        Apply a sharpening filter to the input image.

        Parameters:
        - image: The input image.

        Returns:
        - sharpened_image: The image with sharpening applied.
        """
        kS = np.array([[-1,-1,-1], [-1, 9,-1], [-1,-1,-1]])
        return cv2.filter2D(image, -1, kS)

    @staticmethod
    def tint(image, number, color):
        """
        Apply a color tint to the input image.

        Parameters:
        - image: The input image.
        - number: The tint strength (percentage change).
        - color: The color to tint ("Red," "Green," or "Blue").

        Returns:
        - tinted_image: The tinted image.
        """
        if color == "Red":
            image[:, :, 2] = (image[:, :, 2] * (1 + number / 100)).clip(0,255)
        elif color == "Green":
            image[:, :, 1] = (image[:, :, 1] * (1 + number / 100)).clip(0,255)
        elif color == "Blue":
            image[:, :, 0] = (image[:, :, 0] * (1 + number / 100)).clip(0,255)
        image = image.astype('uint8')
        return image

    @staticmethod
    def flip(image, number):
        """
        Flip the input image horizontally, vertically, or both.

        Parameters:
        - image: The input image.
        - number: -1 for both, 0 for vertical, and 1 for horizontal flip.

        Returns:
        - flipped_image: The flipped image.
        """
        return cv2.flip(image, number)

    @staticmethod
    def rotate(image, angle):
        """
        Rotate the input image by a specified angle.

        Parameters:
        - image: The input image.
        - angle: The rotation angle in degrees.

        Returns:
        - rotated_image: The rotated image.
        """
        rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
        return cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    @staticmethod
    def translate(image, alpha_x, alpha_y):
        """
        Translate (shift) the input image by specified values in the x and y directions.

        Parameters:
        - image: The input image.
        - alpha_x: The translation amount in the x direction.
        - alpha_y: The translation amount in the y direction.

        Returns:
        - translated_image: The translated image.
        """
        translated_image = np.zeros_like(image)
        for y in range(image.shape[0]):
            for x in range(image.shape[1]):
                if 0 <= x + alpha_x < image.shape[1] and 0 <= y + alpha_y < image.shape[0]:
                    translated_image[y + alpha_y, x + alpha_x] = image[y, x]
        return translated_image


In [90]:
def read_config_file(filename):
    with open(filename, "r") as json_file:
        return json.load(json_file)

def store_augmentations(data):
    augmentation_array = []
    for aug_list in data:
        current_element = []
        for augmentation in aug_list:
            current_element.append(AugmentationTechnique(augmentation["type"], augmentation["number1"], augmentation["number2"], augmentation["color"]))
        augmentation_array.append(current_element)
    return augmentation_array

def select_folder():
    global folder_path
    global selected_folder_label
    folder_path = filedialog.askdirectory()
    if folder_path:
        selected_folder_label.config(text=f"Selected Folder: {folder_path}")

def perform_augmentations():
    global augmentations
    global folder_path
    system = AugmentationSystem(augmentations, folder_path)
    system.perform_augmentations()

In [91]:
# Read data from config file and store it
data = read_config_file("config/test_config_1.json")
augmentations = store_augmentations(data)
label_text = "\nAugmentations:\n\n\n"
for aug_list in augmentations:
    for augmentation in aug_list:
        label_text += str(augmentation)
    label_text += "\n\n"

# Create a GUI window
root = tk.Tk()
root.title("Augmentation System")
root.geometry("400x" + str(len(augmentations) * 25 + 250))

augmentation_label = tk.Label(root, text=label_text)
augmentation_label.pack()

selected_folder_label = tk.Label(root, text="Selected folder:")
selected_folder_label.pack()

folder_path = ""

folder_button = tk.Button(root, text="Select Folder", command=select_folder)
folder_button.pack(padx=10, pady=10)

augmentation_button = tk.Button(root, text="Perform augmentations", command=perform_augmentations)
augmentation_button.pack(padx=10, pady=10)

# Start the GUI event loop
root.mainloop()

Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/balloon_Blur_1.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/cat_Blur_2.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/forest_Blur_3.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/house_Blur_4.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/sunset_Blur_5.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/balloon_Sharpen_6.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/cat_Sharpen_7.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/forest_Sharpen_8.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/house_Sharpen_9.jpg
Saved image: C:/Users/Miruna/Desktop/Master/git/Semester 1/FCV/P1/images_aug/sunset_Sharpen_10.jpg
Saved image: C:/Users/Miruna/Desktop/Mas